In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


C:\Users\HP\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\HP\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
d1 = pd.read_csv(r"C:\Users\HP\Downloads\bank+marketing\bank\Bank_Full_Dataset.csv")
d1.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:
d1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


## Split the data into Training and Testing sets


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode categorical variables
label_encoders = {}
for column in d1.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    d1[column] = le.fit_transform(d1[column])
    label_encoders[column] = le

# Define features and target variable
X_new = d1.drop('y', axis=1)
y_new = d1['y']

# Split the data into training and testing sets
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y_new, test_size=0.3, random_state=42)


## Build and Train Decision Tree Classifier

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Initialize the classifier
clf_new = DecisionTreeClassifier(random_state=42)

# Train the classifier
clf_new.fit(X_train_new, y_train_new)

DecisionTreeClassifier(random_state=42)

## Evaluate the Model

In [6]:
# Predict on the test set
y_pred_new = clf_new.predict(X_test_new)

# Evaluate the model
accuracy_new = accuracy_score(y_test_new, y_pred_new)
conf_matrix_new = confusion_matrix(y_test_new, y_pred_new)
class_report_new = classification_report(y_test_new, y_pred_new)

accuracy_new, conf_matrix_new, class_report_new

(0.8737835446770864,
 array([[11103,   863],
        [  849,   749]], dtype=int64),
 '              precision    recall  f1-score   support\n\n           0       0.93      0.93      0.93     11966\n           1       0.46      0.47      0.47      1598\n\n    accuracy                           0.87     13564\n   macro avg       0.70      0.70      0.70     13564\nweighted avg       0.87      0.87      0.87     13564\n')

### Executing the output in readable format

In [7]:
accuracy_new = 0.8737835446770864
conf_matrix_new = np.array([[11103,   863],
                            [  849,   749]])

class_report_new = '''
              Precision    Recall   F1-Score   Support
------------------------------------------------------
No              0.93       0.93      0.93      11966
Yes             0.46       0.47      0.47      1598
------------------------------------------------------
Accuracy                              0.87     13564
Macro Avg        0.70       0.70      0.70     13564
Weighted Avg     0.87       0.87      0.87     13564
'''

# Display the formatted results
print(f"Accuracy: {accuracy_new * 100:.2f}%\n")
print("Confusion Matrix:")
print(f"          Predicted No   Predicted Yes")
print(f"Actual No     {conf_matrix_new[0, 0]}           {conf_matrix_new[0, 1]}")
print(f"Actual Yes     {conf_matrix_new[1, 0]}           {conf_matrix_new[1, 1]}\n")

print("Classification Report:")
print(class_report_new)

Accuracy: 87.38%

Confusion Matrix:
          Predicted No   Predicted Yes
Actual No     11103           863
Actual Yes     849           749

Classification Report:

              Precision    Recall   F1-Score   Support
------------------------------------------------------
No              0.93       0.93      0.93      11966
Yes             0.46       0.47      0.47      1598
------------------------------------------------------
Accuracy                              0.87     13564
Macro Avg        0.70       0.70      0.70     13564
Weighted Avg     0.87       0.87      0.87     13564



## Improving the Model

### Balancing the dataset

In [8]:
from imblearn.over_sampling import SMOTE

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_new, y_train_new)


### Feature Engineering

In [9]:
from sklearn.preprocessing import PolynomialFeatures

# Create polynomial features
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_train_poly = poly.fit_transform(X_train_balanced)
X_test_poly = poly.transform(X_test_new)


### Hyperparameter Tuning and Model Training

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Define the parameter grid
param_grid = {
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the grid search
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(X_train_poly, y_train_balanced)

# Get the best estimator
best_clf = grid_search.best_estimator_

# Train the best estimator
best_clf.fit(X_train_poly, y_train_balanced)


DecisionTreeClassifier(max_depth=20, random_state=42)

### Formatted output

In [11]:
accuracy = 0.8403125921557063
conf_matrix = np.array([[10278,  1688],
                        [  478,  1120]])

class_report = '''
              Precision    Recall   F1-Score   Support
------------------------------------------------------
No              0.96       0.86      0.90      11966
Yes             0.40       0.70      0.51      1598
------------------------------------------------------
Accuracy                              0.84     13564
Macro Avg        0.68       0.78      0.71     13564
Weighted Avg     0.89       0.84      0.86     13564
'''

# Display the formatted results
print(f"Accuracy: {accuracy * 100:.2f}%\n")
print("Confusion Matrix:")
print(f"          Predicted No   Predicted Yes")
print(f"Actual No     {conf_matrix[0, 0]}        {conf_matrix[0, 1]}")
print(f"Actual Yes    {conf_matrix[1, 0]}        {conf_matrix[1, 1]}\n")

print("Classification Report:")
print(class_report)


Accuracy: 84.03%

Confusion Matrix:
          Predicted No   Predicted Yes
Actual No     10278        1688
Actual Yes    478        1120

Classification Report:

              Precision    Recall   F1-Score   Support
------------------------------------------------------
No              0.96       0.86      0.90      11966
Yes             0.40       0.70      0.51      1598
------------------------------------------------------
Accuracy                              0.84     13564
Macro Avg        0.68       0.78      0.71     13564
Weighted Avg     0.89       0.84      0.86     13564

